In [ ]:
import random
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time

In [ ]:
from GetInliersRansac import GetInliersRansac
from EssentialMatrixFromFundamentalMatrix import EssentialMatrixFromFundamentalMatrix
from ExtractCameraPose import ExtractCameraPose
from DisambiguateCameraPose import DisambiguateCameraPose
from LinearTriangulation import LinearTriangulation
from NonLinearTriangulation import NonlinearTriangulation
from LinearPNP import LinearPnP
from PNPRANSAC import PnPRANSAC, AllReprojetionError
from NonLinearPNP import NonLinearPnP
from BundleAdjustment import BundleAdjustment
from Misc.utils import PlotFuncs
from Misc.utils import MiscFuncs


In [ ]:

path='/home/adinisal/adinisal_hw3/Data/Data/'

# loading images
misc_funcs = MiscFuncs()
images = misc_funcs.load_images(path)
# print(images[1])
print("loaded images\n")

# given camera calibration matrix
K = np.array([
            [568.996140852, 0, 643.21055941],
            [0, 568.988362396, 477.982801038],
            [0, 0, 1]
            ])
# define camera 1 as the world pose
M1 = np.identity(4)
M1 = M1[0:3, :]
M1 = np.dot(K, M1)
# for each image pairs compute F, E
'''.............................get inliers and fundamental matrix using RANSAC...........................'''
# load correspondences between image 1 and 2
file_name = "matches12.txt"
print("using correspondences from file "+file_name)
print("performing RANSAC to obtain F matrix\n")
pts_from_txt = misc_funcs.get_pts_from_txt(path, file_name)
pts_from_txt = np.array(pts_from_txt, np.float32)

loaded images

using correspondences from file matches12.txt
performing RANSAC to obtain F matrix



In [ ]:
max_inliers_locs, min_outliers_locs, F_max_inliers, pts_left, pts_right = GetInliersRansac(pts_from_txt)
print("pts_from_txt.shape = ",pts_from_txt.shape)
print("Max inliers.shape = ", max_inliers_locs.shape)

max inliers: {} 879
pts_from_txt.shape =  (1426, 7)
Max inliers.shape =  [[456.25 391.99 308.57 ...  90.    81.    59.  ]
 [456.25 391.99 308.57 ...  90.    81.    59.  ]
 [634.56 276.74 520.96 ... 229.   212.   202.  ]
 ...
 [539.07  53.05 414.95 ... 232.   223.   228.  ]
 [629.51  44.47 513.03 ... 234.   222.   212.  ]
 [577.08  27.16 456.18 ... 233.   249.   253.  ]]


In [14]:
plot_funcs = PlotFuncs()
print("plotting correspondences between images 1 and 2\n")
plot_funcs.plot_img_correspondences(images[0], images[1], max_inliers_locs, min_outliers_locs, file_name, save=True)

plotting correspondences between images 1 and 2



In [17]:
print("estimating E matrx\n")
E = EssentialMatrixFromFundamentalMatrix(F_max_inliers, K)
print("E = ", E)
C2_list, R2_list = ExtractCameraPose(E)
print("C2_List = ", C2_list)
print("R2_List = ", R2_list)

estimating E matrx

E =  [[ 0.02654929  0.73154416  0.32491785]
 [-0.8890472   0.05045153 -0.40888834]
 [-0.20835205  0.57776921  0.14559834]]
C2_List =  [[-0.59880437 -0.19964979  0.77561156]
 [ 0.59880437  0.19964979 -0.77561156]
 [-0.59880437 -0.19964979  0.77561156]
 [ 0.59880437  0.19964979 -0.77561156]]
R2_List =  [[[-0.48756398  0.36911882 -0.79122226]
  [ 0.18538609 -0.841802   -0.50695305]
  [-0.85317839 -0.39385365  0.34200284]]

 [[-0.48756398  0.36911882 -0.79122226]
  [ 0.18538609 -0.841802   -0.50695305]
  [-0.85317839 -0.39385365  0.34200284]]

 [[ 0.97474148  0.06015422 -0.21508258]
  [-0.02295427  0.98492766  0.17143687]
  [ 0.22215343 -0.16216957  0.96143064]]

 [[ 0.97474148  0.06015422 -0.21508258]
  [-0.02295427  0.98492766  0.17143687]
  [ 0.22215343 -0.16216957  0.96143064]]]
